## Project 3
### Task 1
### Data Clustering Based on Human Development Index 

Submitted By: Anish Bhusal (bhusal.anish12@gmail.com)

**Clustering Algorithms used**
1. KMeans
2. DBSCAN
3. Gaussian Mixture Model



Notebook has been organized in following format:

1. Data Cleaning
2. Data Analysis and Visualization
3. Data Pre-processing and feature selection
4. K-Means (Approach Taken and Findings)
5. DBSCAN (Approach Taken and Findings)
6. GMM (Approach Taken and Findings)
7. Conclusion

*Import Necessary Libraries*

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

%matplotlib inline

In [2]:
sns.set(style="whitegrid")

**Read Datasets**

In [3]:
life_expectancy_income=pd.read_excel("../../data/Project-III/life-expectancy-income.xls")
literacy_rate_gender=pd.read_excel("../../data/Project-III/literacy-rates.xls")
education_level=pd.read_csv("../../data/Project-III/literate-population-aged-5-years-and-above-by-educational-attainment.csv")
federal_population=pd.read_csv("../../data/Project-III/total-population-by-sex-country-province-district-and-local-level-population.csv")
federal_units=pd.read_csv("../../data/Project-III/total-number-of-rural-municipality-and-municipality-and-ward-divisions-in-federal-structure-in-2.csv")

### 1. Data Cleaning 


*Let's see what's inside these datasets*

**Cleaning life expectancy and income dataset**

In [4]:
life_expectancy_income.head()

,District,Life expectancy(In Years),Per Capita Income(In USD)
0,Ramechhap,72.90,951
1,Gorkha,71.70,1039
2,Saptari,71.34,801
3,Siraha,71.29,689
4,Rautahat,70.99,757


In [5]:
life_expectancy_income.shape

(75, 3)

In [6]:
life_expectancy_income.dtypes

District                      object
Life expectancy(In Years)    float64
Per Capita Income(In USD)      int64
dtype: object

Check for NaN

In [7]:
life_expectancy_income.isna().sum()

District                     0
Life expectancy(In Years)    0
Per Capita Income(In USD)    0
dtype: int64

Per Capita Income column contains string formatted data. Lets convert it into numerical form

In [8]:
import re 
def clean_per_capita_income(dataset):        
    return int(''.join(re.findall("[0-9]+",dataset)))
life_expectancy_income["Per Capita Income(In USD)"]=life_expectancy_income["Per Capita Income(In USD)"].apply(clean_per_capita_income)
        

TypeError: expected string or bytes-like object

**Cleaning Literacy Rate by Gender dataset**

In [ ]:
literacy_rate_gender.head()

In [ ]:
literacy_rate_gender.shape

In [ ]:
literacy_rate_gender.dtypes

In [ ]:
literacy_rate_gender.isna().sum()

There are no NaN values.

*Year* column has same value i.e. 2013. Dropping the column.

In [ ]:
literacy_rate_gender.drop(columns=[' Year'],inplace=True)

**Cleaning education level for people older than 5 years dataset**

In [ ]:
education_level.head()

In [ ]:
print("Shape of dataset => {}".format(education_level.shape))
print(education_level["Province"].unique)

There are altogether 24 columns and data for 7 provinces and data for whole country

In [ ]:
education_level.isna().sum()

There are no NaN values 

**Cleaning federal population dataset**

In [ ]:
federal_population.head()

In [ ]:
federal_population.shape

In [ ]:
federal_population['year'].unique()

The dataset contains the population of whole nation and by federal level for year 2016 only. 

**Cleaning number of federal units dataset**

In [ ]:
federal_units.head()

In [ ]:
federal_units.dtypes

In [ ]:
federal_units.shape

In [ ]:
federal_units['District'].unique()

The last row contains total of all fed units.

### 2. Data Analysis and Visualization


Let's find maximum and minimum values for life_expectancy and per_capita income

In [ ]:
life_expectancy_income[life_expectancy_income['Life expectancy(In Years)']==max(life_expectancy_income['Life expectancy(In Years)'])]

In [ ]:
life_expectancy_income[life_expectancy_income['Life expectancy(In Years)']==min(life_expectancy_income['Life expectancy(In Years)'])]

Ramechhap is the district having highest life expectancy of 72.9 years whereas Dolpa has least life expectancy of 61.2 years

In [ ]:
life_expectancy_income[life_expectancy_income['Per Capita Income(In USD)']==max(life_expectancy_income["Per Capita Income(In USD)"])]


In [ ]:
life_expectancy_income[life_expectancy_income['Per Capita Income(In USD)']==min(life_expectancy_income["Per Capita Income(In USD)"])]

Manang has highest per capita income of 3166 USD whereas Bajhang has lowest per capita income of 487 USD.

Now let's see how per capita income are distributed in Nepal.

In [ ]:
plt.hist(life_expectancy_income["Per Capita Income(In USD)"])
plt.xlabel("Per Capita Income is USD")
plt.ylabel("Number of Districts")
plt.title("Histogram of Per Capita Income by Districts")
plt.grid(True)
plt.show()

From the distribution plot above, it is clear that most districts have Per Capita Income around 1000 USD

Now let's see how life expectancy is distributed.

In [ ]:
plt.hist(life_expectancy_income["Life expectancy(In Years)"])
plt.xlabel("Life expectancy(In Years)")
plt.ylabel("Number of Districts")
plt.title("Life Expectancy by Districts")
plt.grid(True)
plt.show()

This plot shows that most districts have Life Expectancy of 68-70 years.

In [ ]:
provinces={
    "Province 1":["Bhojpur","Dhankuta","Ilam","Jhapa","Khotang","Morang","Okhaldhunga","Panchthar","Sankhuwasabha","Solukhumbu","Sunsari","Taplejung","Terhathum","Udayapur"],
    "Province 2":["Saptari","Birgunj","Sarlahi","Bara","Siraha","Rautahat","Dhanusha","Mahottari"],
    "Province 3":["Sindhuli","Ramechhap","Dolakha","Bhaktapur","Dhading","Kathmandu","Kavrepalanchowk","Lalitpur","Nuwakot","Rasuwa","Sidhupalchok","Chitwan","Makwanpur"],
    "Gandaki":["Baglung","Gorkha","Kaski","Lamjung","Manang","Mustang","Myagdi","Nawalpur","Parbat","Syangja","Tanahun"],
    "Province 5":["Kapilvastu","Parasi","Rupandehi","Arghakhanchi","Gulmi","Palpa","Dang","Pyuthan","Rolpa","Eastern Rukum","Banke","Bardiya"],
    "Karnali":["Western Rukum","Salyan","Dolpa","Humla","Jumla","Kalikot","Mugu","Surkhet","Dailekh","Jajarkot"],
    "Province 7":["Kailali","Achham","Doti","Bajhang","Bajura","Kanchanpur","Dadeldhura","Baitadi","Darchula"]
    }

Now let's analyze literacy rate dataset.

In [ ]:
literacy_rate_gender[literacy_rate_gender["Total"]==max(literacy_rate_gender["Total"])]

In [ ]:
literacy_rate_gender[literacy_rate_gender["Total"]==min(literacy_rate_gender["Total"])]

Kathmandu has highest literacy rate of 86.3% and Rautahat has least literacy rate of only 41.7 %

In [ ]:
plt.hist(literacy_rate_gender["Male"])
plt.xlabel("Literacy rate of Male")
plt.ylabel("Number of Districts")
plt.title("Literacy Rate Distribution of Male Population")
plt.grid(True)
plt.show()

In [ ]:
plt.hist(literacy_rate_gender["Female"])
plt.xlabel("Literacy rate of Female")
plt.ylabel("Number of Districts")
plt.title("Literacy Rate Distribution of Female Population")
plt.grid(True)
plt.show()

In [ ]:
plt.hist(literacy_rate_gender["Total"])
plt.xlabel("Literacy rate")
plt.ylabel("Number of Districts")
plt.title("Literacy Rate Distribution of Total Population by Districts")
plt.grid(True)
plt.show()

The graphs show the distribution of Literacy Rate by Gender among districts

In [ ]:
literacy_rate_provinces={}
for province,districts in provinces.items():
    literacy_rate_provinces[province]=[0,0, 0]
    for dist in districts:
        l_rate=literacy_rate_gender[literacy_rate_gender["District"].str.match(dist)]["Total"]
        if l_rate.empty:
            continue
        rate=l_rate.values[0] 
        m_rate=(literacy_rate_gender[literacy_rate_gender["District"].str.match(dist)]["Male"]).values[0]
        f_rate=(literacy_rate_gender[literacy_rate_gender["District"].str.match(dist)]["Female"]).values[0]
        if rate>literacy_rate_provinces[province][0]:
            literacy_rate_provinces[province][0]=rate
        if m_rate>literacy_rate_provinces[province][1]:
            literacy_rate_provinces[province][1]=m_rate
        if m_rate>literacy_rate_provinces[province][2]:
            literacy_rate_provinces[province][2]=f_rate

In [ ]:
provinces_df=pd.DataFrame(data=literacy_rate_provinces.keys(),columns=["Provinces"])

In [ ]:
rates=literacy_rate_provinces.values()
total_rate=[x[0] for x in rates]
male_rate=[x[1] for x in rates]
female_rate=[x[2] for x in rates]
provinces_df["Total Literacy"]=total_rate
provinces_df["Male Literacy"]=male_rate
provinces_df["Female Literacy"]=female_rate


In [ ]:
provinces_df

In [ ]:
plt.figure(figsize=(12,8))
barWidth = 0.25

r1 = np.arange(len(total_rate))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]

plt.bar(r1,total_rate,width=barWidth, edgecolor='white', label='Total')
plt.bar(r2,male_rate,width=barWidth, edgecolor='white', label='Male')
plt.bar(r3,female_rate,width=barWidth, edgecolor='white', label='Female')


plt.xlabel('Province')
plt.xticks([r + barWidth for r in range(len(total_rate))], ['Province 1', 'Province 2', 'Province 3', 'Gandaki', 'Province 5','Karnali','Province 7'])
plt.ylabel("Literacy Rate")
plt.title("Literacy rates by Province level")

plt.legend()
plt.show()


This plot shows that Province 3 has highest literacy rate whereas Province 2 has least literacy rate among all 7 provinces.

Also male literacy rate is higher in Province 3 and female literacy rate is higher in Gandaki Province.
Similary both are lowest in Province 2. 

### 3. Data Pre-Processing and Feature Selection


As part of data pre-processing, combine literacy rate dataset and life expectancy dataset 

In [ ]:
final_df=pd.concat([life_expectancy_income,literacy_rate_gender[["Total"]]],axis=1)

In [ ]:
final_df.head()

In [ ]:
#rename total to literacy rate
final_df.rename({"Total":"Literacy Rate"},axis='columns',inplace=True)

In [ ]:
final_df.columns

**Normalize Data**

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler=StandardScaler()
X=final_df.drop(columns=['District'])
scaler.fit(X)
final_scaled_data=scaler.transform(X)
final_scaled_data.shape

### 4. KMeans 

In [ ]:
from sklearn.cluster import KMeans

**Choosing number of clusters using Elbow method**

In [ ]:
inertia=[]
num_of_clusters=[]
for k in range(2,10):
    kmeans=KMeans(n_clusters=k,random_state=1)
    pred=kmeans.fit(X)
    inertia.append(kmeans.inertia_)
    num_of_clusters.append(k)
    
plt.plot(num_of_clusters,inertia)
plt.grid()
plt.xlabel("Different values of k (num of clusters)")
plt.ylabel("Inertia")
plt.title("Elbow method for choosing the value of k")
plt.show()

From above graph, num of clusters=3 can be choose for further steps

In [ ]:
kmeans=KMeans(n_clusters=3,random_state=1)
kmeans.fit(final_scaled_data)


In [ ]:
kmeans.labels_

In [ ]:
kmeans.cluster_centers_

**Using PCA for visualizing cluster**

In [ ]:
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(final_scaled_data)

In [ ]:
COLOR_MAP = {0 : 'r',
                   1 : 'g',
                   2 : 'b'}

label_color = [COLOR_MAP[l] for l in kmeans.labels_]
plt.figure(figsize=(12,8))
plt.scatter(principalComponents[:,0],principalComponents[:,1],c=label_color, alpha=0.9)
plt.title("Cluster visualization of HDI data using KMeans")
plt.show()

In [ ]:
silhouette_score(principalComponents,kmeans.labels_)

For 3 clusters, the silhouette score is 0.44

### 5. DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN

Choosing the value of epison and min_samples

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
neigh=NearestNeighbors(n_neighbors=9)
nbrs=neigh.fit(final_scaled_data)
distances,indices=nbrs.kneighbors(final_scaled_data)

In [ ]:
distances=np.sort(distances,axis=0)
distances_=distances[:,8]
plt.plot(distances_)

In [ ]:
distances.shape

Choosing eps=2 from the above graph

In [ ]:
dbscan=DBSCAN(eps=2.,min_samples=3,metric='euclidean')
dbscan.fit(final_scaled_data)

In [ ]:
dbscan.labels_

In [ ]:
plt.figure(figsize=(12,8))
COLOR_MAP = {-1 : 'r',
                   0 : 'g',
                   1 : 'b'}

label_color = [COLOR_MAP[l] for l in dbscan.labels_]
plt.scatter(final_scaled_data[:, 0], final_scaled_data[:, 1], c=label_color,alpha=0.9);
plt.title("Scatter plot for DBSCAN")
plt.show()

DBSCAN formed only a single cluster.

In [ ]:
silhouette_score(final_scaled_data,dbscan.labels_)

For eps=2 and min_samples=8, DBSCAN received 0.588 sihouette score.

## 6. GMM 

In [ ]:
from sklearn.mixture import GaussianMixture


KMeans can be used to select select number of components. 

In [ ]:
gmm=GaussianMixture(n_components=3)
gmm.fit(final_scaled_data)

In [ ]:
labels=gmm.predict(final_scaled_data)
labels

In [ ]:
plt.figure(figsize=(12,8))
COLOR_MAP = {0 : 'r',
                   1 : 'g',
                   2 : 'b'}

label_color = [COLOR_MAP[l] for l in labels]
plt.scatter(final_scaled_data[:, 0], final_scaled_data[:, 1], c=label_color,alpha=0.9);
plt.title("Scatter plot for GMM")
plt.show()

NOw calculate Silhouette Score for clusters formed

In [ ]:
silhouette_score(final_scaled_data,labels)

## 7. Conclusion

Three models were used for clustering HDI dataset on parameters: Life Expectancy, Per Capita Income and Literacy Rate.
On evaulating the performance of all models, it can be conlcuded that Gaussian Mixture Model with number of components=3 performed
well than other model achieveing Silhouette score of 0.4484